# FAA/Esri Airports NOTAM Status Living Atlas Layer 

**June 20th, 2017**

**Version 0.1**

*Documentation and Guidelines by Matthew Thompson, FAA*

*Notebook, Scripted Process, and Testing by Alberto Nieto, Esri*


### Purpose:
To help fulfill FAA objectives by outlining potential data sources and techniques to generate a real-time, data rich layer within Esri’s Living Atlas using the Python API for ArcGIS.

### End Goal:
The end goal of the project should be to have all ICAO coded public use airports in the United States as points available in Living Atlas as a feature service. Weather and NOTAM information should be available as attribute data to each airport. Additionally, NOTAM data should be parsed to populate an additional attribute field for “Airport NOTAM Status” indicating if the airport has a NOTAM issued that may affect the status of the airport.

### Data Parsing

#### National Flight Data Center (NFDC)
NFDC data is updated every 28 days and available for download as a fixed field width text file. File reference documentation is available on the NFDC website. This project will only make use of “APT” records in the APT file. Only ICAO coded, public use airports will be used in this phase. The following pseudo code should provide the necessary output to map the desired airports. 

~~~~
Select LOCATION IDENTIFIER, ICAO IDENTIFIER, OFFICIAL FACILITY NAME, AIRPORT REFERENCE POINT LATITUDE (FORMATTED), AIRPORT REFERENCE POINT LONGITUDE (FORMATTED)
From NFDC APT File
Where (
LANDING FACILITY TYPE = “Airport” AND
FACILITY USE = “PU” AND
ICAO IDENTIFIER IS NOT NULL)
~~~~

Of course, any additional information that may be useful can also be retained in the layer. Output of the above latitude and longitude will be used for feature position in the layer.

#### Aviation Digital Data Service (METARs)
METAR information should be added to the above features using ADDS data. METAR information is generated hourly. There are multiple search criteria available on the ADDS website. The stationString on in ADDS corresponds to the ICAO Identifier of the feature earlier identified. It is preferred that METAR information is available as both current and previous in the attribute data, in separate fields.

#### Aviation Digital Data Service (TAFs):
TAF information should be added to the above features using ADDS data. TAF information is generated daily. There are multiple search criteria available on the ADDS website. The stationString on in ADDS corresponds to the ICAO Identifier of the feature earlier identified. Only the current TAF is required.

#### Aeronautical Information Data Access Portal (AIDAP) Notice to Airmen (NOTAM) Data:
NOTAM information should be added to the above features using AIDAP data procured in the development teams preferred fashion (Java or WS). Association of the AIDAP provided NOTAMs should be based upon ICAO Identifier of the feature earlier identified. NOTAM information is purely text and can be several dozen messages, and may need to be handled in multiple fields, or in a fashion other than pure attribute of the feature. Examples of NOTAM information can be found by visiting https://pilotweb.nas.faa.gov and entering ICAO Identifiers in the “NOTAM Retrieval”, “Locations” section and clicking “View NOTAMs”.

#### Parsing NOTAMs for closure:
NOTAMs contain valuable information about the status of the airport in question. It would be exceptionally beneficial if the layer had an automatically derived Airport NOTAM Status field available. The proposed values for this field are:
•	Open
•	Closed
•	Limited

This is exceptionally complicated. The key term to search for in NOTAMs to identify if they will trigger this status is “AD AP CLSD”, however, the presence of “AD AP CLSD” does not necessarily trigger the field to full “Closed” status. The text immediately following “AD AP CLSD” will dictate if the status field receives a “Closed” or “Limited” status.

If, immediately following “AD AP CLSD” there is a space followed by a numeric string ten characters long, the airport receives a “Closed” status.

If, immediately following “AD AP CLSD” there is anything other than the previously mentioned string, the airport receives a “Limited” status.

#### Examples:

NOTAM:
!AGS 03/034 AGS AD AP CLSD TO FREIGHT, MEDICAL, AIR CARGO, AND FORMATION FLT 1704031200-1704112200
Automatic Status: Limited

NOTAM:
!WYS 11/004 WYS AD AP CLSD EXC LIFE FLT HEL 1611172345-1705152359EST
Automatic Status: Limited

NOTAM:
!FDR 03/002 FDR AD AP CLSD 1703281316-1704122359
Automatic Status: Closed

NOTAM:
!TEX 03/023 TEX AD AP CLSD 1704030300-1706251300
Automatic Status: Closed

An airport without a NOTAM containing “AD AP CLSD” receives the automatic status of “Open”.

## Execution of Scripted Process

#### Connect to a GIS

In [42]:
# Establish my GIS
import arcgis
from IPython.display import display
import getpass

In [54]:
portals_dict = {
    "esrifederal_gis": r"https://esrifederal.maps.arcgis.com",
    "natgov_gis": r"http://esri-natgov105.eastus.cloudapp.azure.com/arcgis",
    "dot_gis": r"http://dot.esri.com/portal",
    "dev_gis": r"http://govdev.eastus.cloudapp.azure.com/arcgis",
    "local_gis": r"https://anieto.esri.com/arcgis"
}

In [55]:
gis_url = portals_dict["dev_gis"]

In [56]:
gis_username = getpass.getpass(prompt="Username: ")

Username: ········


In [57]:
gis_pw = getpass.getpass(prompt="Password: ")

Password: ········


In [62]:
print("Attempting to log in to '{0}'...".format(gis_url))
gis = arcgis.gis.GIS(gis_url, gis_username, gis_pw, verify_cert=False)
print("Successfully logged in as: " + gis.properties.user.username)

Attempting to log in to 'http://govdev.eastus.cloudapp.azure.com/arcgis'...
Successfully logged in as: anieto


In [61]:
gis.map()

### Core Data Download and Parsing: NFDC APT

NFDC data is updated every 28 days and available for download as a fixed field width text file. File reference documentation is available on the NFDC website. This project will only make use of “APT” records in the APT file. Only ICAO coded, public use airports will be used in this phase. The following pseudo code should provide the necessary output to map the desired airports. 

~~~~
Select LOCATION IDENTIFIER, ICAO IDENTIFIER, OFFICIAL FACILITY NAME, AIRPORT REFERENCE POINT LATITUDE (FORMATTED), AIRPORT REFERENCE POINT LONGITUDE (FORMATTED)
From NFDC APT File
Where (
LANDING FACILITY TYPE = “Airport” AND
FACILITY USE = “PU” AND
ICAO IDENTIFIER IS NOT NULL)
~~~~

Of course, any additional information that may be useful can also be retained in the layer. Output of the above latitude and longitude will be used for feature position in the layer.

In [63]:
# Import needed modules to download the APT file
import requests
import zipfile
import io

In [9]:
# Test of loading of NFDC-APT file
apt_url = r"https://nfdc.faa.gov/webContent/28DaySub/2017-06-22/APT.zip"
inputs_dir = r"C:\Users\albe9057\Documents\ANieto_SolutionEngineering\Projects\DOT\FAA\FAA_LivingAtlas_Layer\Inputs\NFDC_NASR_APT"

In [10]:
# Download and unzip the apt_zip
response = requests.get(apt_url)
zipDocument = zipfile.ZipFile(io.BytesIO(response.content))
zipDocument.extractall(path=inputs_dir)
apt_file = "{0}\\{1}".format(inputs_dir, "apt.txt")

In [11]:
apt_file

'C:\\Users\\albe9057\\Documents\\ANieto_SolutionEngineering\\Projects\\DOT\\FAA\\FAA_LivingAtlas_Layer\\Inputs\\NFDC_NASR_APT\\apt.txt'

#### Set APT Schema
Schema documentation for APT fixed-width-file from FAA found at: 
https://nfdc.faa.gov/webContent/28DaySub/2017-06-22/Layout_Data/apt_rf.txt

In [12]:
apt_file_schema = {
    "apt": {
        "record_type_indicator": (0, 3),
        "landing_facility_site_number": (3, 14),
        "landing_facility_type": (14, 27),
        "location_identifier": (27, 31),
        "information_effective_date": (31, 41),
        "faa_region_code": (41, 44),
        "faa_district_or_field_office_code": (44, 48),
        "associated_state_post_office_code": (48, 50),
        "associated_state_name": (50, 70),
        "associated_county_name": (70, 91),
        "associated_county_state": (91, 93),
        "associated_city_name": (93, 133),
        "official_facility_name": (133, 183),
        "airport_ownership_type": (183, 185),
        "facility_use": (185, 187),
        "facility_owner_name": (187, 222),
        "owner_address": (222, 294),
        "owner_city_state_zip": (294, 339),
        "owner_phone_number": (339, 355),
        "facility_manager_name": (355, 390),
        "manager_address": (390, 462),
        "manager_city_state_zip": (462, 507),
        "manager_phone_number": (507, 523),
        "airport_reference_point_latitude_formatted": (523, 538),
        "airport_reference_point_latitude_seconds": (538, 550),
        "airport_reference_point_longitude_formatted": (550, 565),
        "airport_reference_point_longitude_seconds": (565, 577),
        "airport_reference_point_determination_method": (577, 578),
        "airport_elevation": (578, 585),
        "airport_elevation_determination_method": (585, 586),
        "magnetic_variation_and_direction": (586, 589),
        "magnetic_variation_epoch_year": (589, 593),
        "traffic_pattern_altitude": (593, 597),
        "aeronautical_sectional_chart": (597, 627),
        "distance_from_cbd_of_associated_city": (627, 629),
        "land_area_covered_by_airport": (632, 637),
        "boundary_artcc_identifier": (637, 641),
        "boundary_artcc_computer_identifier": (641, 644),
        "boundary_artcc_name": (644, 674),
        "responsible_artcc_identifier": (674, 678),
        "responsible_artcc_computer_identifier": (678, 681),
        "responsible_artcc_name": (680, 711),
        "tiein_fss_physically_located_on_facility": (711, 712),
        "tiein_fss_identifier": (712, 716),
        "tiein_fss_name": (715, 746),
        "local_phone_number_from_airport_to_fss": (746, 762),
        "tollfree_phone_number_from_airport_to_fss": (762, 778),
        "alternate_fss_identifier": (778, 782),
        "alternate_fss_name": (782, 812),
        "tollfree_phone_number_from_airport_to_altfss": (812, 828),
        "identifier_of_facility_resp_for_notams_and_weather": (828, 832),
        "availability_of_notam_d_service": (832, 833),
        "airport_activation_date": (833, 840),
        "airport_status_code": (840, 842),
        "airport_arff_cert_type_and_date": (842, 857),
        "npias_federal_agreements_code": (857, 864),
        "airport_airspace_analysis_determination": (864, 877),
        "ustreasury_desig_as_interntl_entry_for_customs": (877, 878),
        "ustreasury_desig_as_customs_landing_rights": (878, 879),
        "has_military_civil_joint_use_agrmnt": (879, 880),
        "airport_has_agrmnt_grants_military_landing_rights": (880, 881),
        "airport_inspection_method": (881, 883),
        "agency_performing_physical_inspection": (883, 884),
        "last_physical_inspection_date": (884, 892),
        "last_date_information_request_completed_by_mgr": (892, 900),
        "fuel_types_available": (900, 940),
        "airframe_repair_service_availability": (940, 945),
        "powerplant_engine_repair_availability": (945, 950),
        "type_of_bottled_oxygen_available": (950, 958),
        "type_of_bulk_oxygen_available": (958, 966),
        "airport_lighting_schedule": (966, 973),
        "beacon_lighting_schedule": (973, 980),
        "air_traffic_control_twr_located_on_airport": (980, 981),
        "unicom_freq_available_at_airport": (981, 988),
        "common_traffic_advisory_freq": (988, 995),
        "segmtd_circle_airport_marker_system": (995, 999),
        "lens_color_of_operable_beacon": (999, 1002),
        "landing_fee_charged_to_noncommercial_users": (1002, 1003),
        "used_for_medical_purposes": (1002, 1004),
        "single_engine_gen_aviation_aircraft": (1004, 1007),
        "multi_engine_gen_aviation_aircraft": (1007, 1010),
        "jet_enging_gen_aviation_aircraft": (1010, 1013),
        "gen_aviation_helicopter": (1013, 1016),
        "operational_gliders": (1016, 1019),
        "operational_military_aircraft": (1019, 1022),
        "ultralight_aircraft": (1022, 1025),
        "commercial_services": (1025, 1031),
        "commuter_services": (1031, 1037),
        "air_taxi": (1037, 1043),
        "gen_aviation_local_ops": (1043, 1049),
        "gen_aviation_itinerant_ops": (1049, 1055),
        "military_aircraft_ops": (1055, 1061),
        "reference_ending_date_for_ops_fields": (1061, 1071),
        "airport_position_source": (1071, 1087),
        "airport_position_source_date": (1087, 1097),
        "airport_elev_source": (1097, 1113),
        "airport_elev_source_date": (1113, 1123),
        "contract_fuel_available": (1123, 1124),
        "transient_storage_facilities": (1124, 1136),
        "other_airport_services_available": (1136, 1207),
        "wind_indicator": (1207, 1210),
        "icao_identifier": (1210, 1217),
        "airport_record_filler": (1217, 1218)
    }
}

In [13]:
# To get the specific record type of field character locations as a list:
apt_col_specs = list(apt_file_schema["apt"].values())
apt_col_specs

[(0, 3),
 (3, 14),
 (14, 27),
 (27, 31),
 (31, 41),
 (41, 44),
 (44, 48),
 (48, 50),
 (50, 70),
 (70, 91),
 (91, 93),
 (93, 133),
 (133, 183),
 (183, 185),
 (185, 187),
 (187, 222),
 (222, 294),
 (294, 339),
 (339, 355),
 (355, 390),
 (390, 462),
 (462, 507),
 (507, 523),
 (523, 538),
 (538, 550),
 (550, 565),
 (565, 577),
 (577, 578),
 (578, 585),
 (585, 586),
 (586, 589),
 (589, 593),
 (593, 597),
 (597, 627),
 (627, 629),
 (632, 637),
 (637, 641),
 (641, 644),
 (644, 674),
 (674, 678),
 (678, 681),
 (680, 711),
 (711, 712),
 (712, 716),
 (715, 746),
 (746, 762),
 (762, 778),
 (778, 782),
 (782, 812),
 (812, 828),
 (828, 832),
 (832, 833),
 (833, 840),
 (840, 842),
 (842, 857),
 (857, 864),
 (864, 877),
 (877, 878),
 (878, 879),
 (879, 880),
 (880, 881),
 (881, 883),
 (883, 884),
 (884, 892),
 (892, 900),
 (900, 940),
 (940, 945),
 (945, 950),
 (950, 958),
 (958, 966),
 (966, 973),
 (973, 980),
 (980, 981),
 (981, 988),
 (988, 995),
 (995, 999),
 (999, 1002),
 (1002, 1003),
 (1002, 1

In [14]:
apt_col_names = list(apt_file_schema["apt"].keys())
apt_col_names

['record_type_indicator',
 'landing_facility_site_number',
 'landing_facility_type',
 'location_identifier',
 'information_effective_date',
 'faa_region_code',
 'faa_district_or_field_office_code',
 'associated_state_post_office_code',
 'associated_state_name',
 'associated_county_name',
 'associated_county_state',
 'associated_city_name',
 'official_facility_name',
 'airport_ownership_type',
 'facility_use',
 'facility_owner_name',
 'owner_address',
 'owner_city_state_zip',
 'owner_phone_number',
 'facility_manager_name',
 'manager_address',
 'manager_city_state_zip',
 'manager_phone_number',
 'airport_reference_point_latitude_formatted',
 'airport_reference_point_latitude_seconds',
 'airport_reference_point_longitude_formatted',
 'airport_reference_point_longitude_seconds',
 'airport_reference_point_determination_method',
 'airport_elevation',
 'airport_elevation_determination_method',
 'magnetic_variation_and_direction',
 'magnetic_variation_epoch_year',
 'traffic_pattern_altitude',

In [15]:
len(apt_col_names)

103

#### NFDC APT Data Parsing and Loading to GIS

In [16]:
import pandas as pd

In [17]:
df = pd.read_fwf(apt_file, header=None, names=apt_col_names, colspecs=apt_col_specs)

In [18]:
df.head()

,record_type_indicator,landing_facility_site_number,landing_facility_type,location_identifier,information_effective_date,faa_region_code,faa_district_or_field_office_code,associated_state_post_office_code,associated_state_name,associated_county_name,...,airport_position_source,airport_position_source_date,airport_elev_source,airport_elev_source_date,contract_fuel_available,transient_storage_facilities,other_airport_services_available,wind_indicator,icao_identifier,airport_record_filler
0,APT,50009.*A,AIRPORT,ADK,06/22/2017,AAL,NONE,AK,ALASKA,ALEUTIANS WEST,...,3RD PARTY SURVEY,07/17/2011,3RD PARTY SURVEY,07/17/2011,NaN,HGR,CARGO,Y,PADK,NaN
1,ATT,50009.*A,AK 1ALL/ALL/0,800-,1600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWY,50009.*A,AK05/23 779,0 20,0ASPH-F,NaN,GRVD,4,9 /R/B/X/THIGH 05 06,1 YNPI G51-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RMK,50009.*A,AKA110-11,PS,BL WIND SH,EAR,S LN,DG,/DEPARTING ALL RYS.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RMK,50009.*A,AKA110-14,AL,L RWYS GRV,D;,DO N,OT,MAKE LOCKED-BRAKE T,URNS.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.shape

(139822, 103)

In [20]:
# Parse the file for records beginning with "APT*"
apt_df = df.loc[df["record_type_indicator"].str.contains("APT")]
del df

In [21]:
apt_df.head()

,record_type_indicator,landing_facility_site_number,landing_facility_type,location_identifier,information_effective_date,faa_region_code,faa_district_or_field_office_code,associated_state_post_office_code,associated_state_name,associated_county_name,...,airport_position_source,airport_position_source_date,airport_elev_source,airport_elev_source_date,contract_fuel_available,transient_storage_facilities,other_airport_services_available,wind_indicator,icao_identifier,airport_record_filler
0,APT,50009.*A,AIRPORT,ADK,06/22/2017,AAL,NONE,AK,ALASKA,ALEUTIANS WEST,...,3RD PARTY SURVEY,07/17/2011,3RD PARTY SURVEY,07/17/2011,NaN,HGR,CARGO,Y,PADK,NaN
20,APT,50016.1*A,AIRPORT,AKK,06/22/2017,AAL,NONE,AK,ALASKA,KODIAK ISLAND,...,NACO,03/18/1997,NACO,03/18/1997,NaN,NaN,NaN,Y,PAKH,NaN
29,APT,50017.*A,AIRPORT,Z13,06/22/2017,AAL,NONE,AK,ALASKA,BETHEL,...,STATE,12/02/2013,STATE,12/02/2013,NaN,NaN,CARGO,Y-L,NaN,NaN
37,APT,50017.1*C,SEAPLANE BASE,KKI,06/22/2017,AAL,NONE,AK,ALASKA,BETHEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN
49,APT,50020.*A,AIRPORT,AKI,06/22/2017,AAL,NONE,AK,ALASKA,BETHEL,...,STATE,09/15/2014,STATE,09/15/2014,NaN,NaN,CARGO,Y-L,PFAK,NaN


In [22]:
apt_df.loc[0]

record_type_indicator                                                       APT
landing_facility_site_number                                           50009.*A
landing_facility_type                                                   AIRPORT
location_identifier                                                         ADK
information_effective_date                                           06/22/2017
faa_region_code                                                             AAL
faa_district_or_field_office_code                                          NONE
associated_state_post_office_code                                            AK
associated_state_name                                                    ALASKA
associated_county_name                                           ALEUTIANS WEST
associated_county_state                                                      AK
associated_city_name                                                ADAK ISLAND
official_facility_name                  

In [23]:
apt_df.shape

(19765, 103)

##### Needed query:
~~~~
Select LOCATION IDENTIFIER, ICAO IDENTIFIER, OFFICIAL FACILITY NAME, AIRPORT REFERENCE POINT LATITUDE (FORMATTED), AIRPORT REFERENCE POINT LONGITUDE (FORMATTED)
From NFDC APT File
Where (
LANDING FACILITY TYPE = “Airport” AND
FACILITY USE = “PU” AND
ICAO IDENTIFIER IS NOT NULL)
~~~~

In [24]:
fields_needed = ["location_identifier", 
                 "icao_identifier", 
                 "official_facility_name", 
                 "airport_reference_point_latitude_formatted", 
                 "airport_reference_point_longitude_formatted"]

In [25]:
apt_df.landing_facility_type.unique()

array(['AIRPORT', 'SEAPLANE BASE', 'HELIPORT', 'GLIDERPORT', 'ULTRALIGHT',
       'BALLOONPORT'], dtype=object)

In [26]:
apt_df.facility_use.unique()

array(['PU', 'PR'], dtype=object)

In [27]:
apt_filtered_df = apt_df[(apt_df.landing_facility_type == "AIRPORT") & (apt_df.facility_use == "PU") & (apt_df.icao_identifier.notnull())]

In [28]:
apt_df.shape

(19765, 103)

In [29]:
apt_filtered_df.shape

(2439, 103)

In [30]:
apt_filtered_df

,record_type_indicator,landing_facility_site_number,landing_facility_type,location_identifier,information_effective_date,faa_region_code,faa_district_or_field_office_code,associated_state_post_office_code,associated_state_name,associated_county_name,...,airport_position_source,airport_position_source_date,airport_elev_source,airport_elev_source_date,contract_fuel_available,transient_storage_facilities,other_airport_services_available,wind_indicator,icao_identifier,airport_record_filler
0,APT,50009.*A,AIRPORT,ADK,06/22/2017,AAL,NONE,AK,ALASKA,ALEUTIANS WEST,...,3RD PARTY SURVEY,07/17/2011,3RD PARTY SURVEY,07/17/2011,NaN,HGR,CARGO,Y,PADK,NaN
20,APT,50016.1*A,AIRPORT,AKK,06/22/2017,AAL,NONE,AK,ALASKA,KODIAK ISLAND,...,NACO,03/18/1997,NACO,03/18/1997,NaN,NaN,NaN,Y,PAKH,NaN
49,APT,50020.*A,AIRPORT,AKI,06/22/2017,AAL,NONE,AK,ALASKA,BETHEL,...,STATE,09/15/2014,STATE,09/15/2014,NaN,NaN,CARGO,Y-L,PFAK,NaN
67,APT,50022.1*A,AIRPORT,7AK,06/22/2017,AAL,NONE,AK,ALASKA,ALEUTIANS EAST,...,STATE,11/17/2014,3RD PARTY SURVEY,10/03/2012,NaN,NaN,NaN,Y-L,PAUT,NaN
76,APT,50024.1*A,AIRPORT,AUK,06/22/2017,AAL,NONE,AK,ALASKA,WADE HAMPTON,...,3RD PARTY SURVEY,12/14/2012,3RD PARTY SURVEY,12/14/2012,NaN,TIE,CARGO,Y-L,PAUK,NaN
127,APT,50029.1*A,AIRPORT,6A8,06/22/2017,AAL,NONE,AK,ALASKA,YUKON-KOYUKUK,...,3RD PARTY SURVEY,07/02/2005,3RD PARTY SURVEY,07/02/2005,NaN,TIE,NaN,Y-L,PFAL,NaN
138,APT,50029.61*A,AIRPORT,AFM,06/22/2017,AAL,NONE,AK,ALASKA,NORTHWEST ARCTIC,...,FAA,09/25/2015,3RD PARTY SURVEY,06/17/2012,NaN,TIE,CHTR,Y-L,PAFM,NaN
163,APT,50032.*A,AIRPORT,AKP,06/22/2017,AAL,NONE,AK,ALASKA,YUKON-KOYUKUK,...,3RD PARTY SURVEY,06/20/2012,3RD PARTY SURVEY,06/20/2012,NaN,TIE,NaN,N,PAKP,NaN
259,APT,50034.*A,AIRPORT,ANC,06/22/2017,AAL,NONE,AK,ALASKA,ANCHORAGE,...,3RD PARTY SURVEY,04/09/2012,3RD PARTY SURVEY,04/09/2012,NaN,"HGR,TIE",CARGO,Y-L,PANC,NaN
284,APT,50035.*A,AIRPORT,MRI,06/22/2017,AAL,NONE,AK,ALASKA,ANCHORAGE,...,FAA-EST,04/28/2011,NACO,07/09/2003,NaN,"HGR,TIE","AFRT,AMB,AVNCS,CARGO,CHTR,INSTR,RNTL,SALES,SURV",Y-L,PAMR,NaN


In [308]:
apt_final_df = apt_filtered_df[fields_needed]

In [309]:
apt_final_df

,location_identifier,icao_identifier,official_facility_name,airport_reference_point_latitude_formatted,airport_reference_point_longitude_formatted
0,ADK,PADK,ADAK,51-53-00.8954N,176-38-32.9277W
20,AKK,PAKH,AKHIOK,56-56-19.2870N,154-10-57.2000W
49,AKI,PFAK,AKIAK,60-54-10.4300N,161-13-50.1000W
67,7AK,PAUT,AKUTAN,54-08-40.6000N,165-36-14.7900W
76,AUK,PAUK,ALAKANUK,62-40-59.0000N,164-43-19.9000W
127,6A8,PFAL,ALLAKAKET,66-33-06.6000N,152-37-19.8000W
138,AFM,PAFM,AMBLER,67-06-22.3282N,157-51-25.8151W
163,AKP,PAKP,ANAKTUVUK PASS,68-08-01.0000N,151-44-36.0000W
259,ANC,PANC,TED STEVENS ANCHORAGE INTL,61-10-26.9634N,149-59-53.4791W
284,MRI,PAMR,MERRILL FIELD,61-12-48.7622N,149-50-41.0069W


#### Format Latitude and Longitude Values

Helper functions for conversion of decimal minutes seconds to decimal degrees

In [310]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'E' or direction == 'N':
        dd *= -1
    return dd

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])

    return (lat)

# Establish a quick helper function to calculate negative values across numpy/pandas array
def convert_to_neg(longitude_string):
    return "-{0}".format(longitude_string)

In [311]:
test_lat = "60-54-10.4300N"

In [312]:
parse_dms(test_lat)

60.90277777777778

In [313]:
apt_final_df["latitude_dd"] = apt_final_df.apply(lambda x: parse_dms(x["airport_reference_point_latitude_formatted"]), axis=1)

C:\Users\albe9057\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [314]:
apt_final_df["longitude_dd"] = apt_final_df.apply(lambda x: parse_dms(x["airport_reference_point_longitude_formatted"]), axis=1)

C:\Users\albe9057\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [315]:
apt_final_df["longitude_dd"] = apt_final_df.apply(lambda x: convert_to_neg(x["longitude_dd"]), axis=1)

C:\Users\albe9057\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [316]:
apt_final_df

,location_identifier,icao_identifier,official_facility_name,airport_reference_point_latitude_formatted,airport_reference_point_longitude_formatted,latitude_dd,longitude_dd
0,ADK,PADK,ADAK,51-53-00.8954N,176-38-32.9277W,51.883333,-176.64222222222222
20,AKK,PAKH,AKHIOK,56-56-19.2870N,154-10-57.2000W,56.938611,-154.1825
49,AKI,PFAK,AKIAK,60-54-10.4300N,161-13-50.1000W,60.902778,-161.23055555555555
67,7AK,PAUT,AKUTAN,54-08-40.6000N,165-36-14.7900W,54.144444,-165.6038888888889
76,AUK,PAUK,ALAKANUK,62-40-59.0000N,164-43-19.9000W,62.683056,-164.72194444444446
127,6A8,PFAL,ALLAKAKET,66-33-06.6000N,152-37-19.8000W,66.551667,-152.62194444444447
138,AFM,PAFM,AMBLER,67-06-22.3282N,157-51-25.8151W,67.106111,-157.85694444444445
163,AKP,PAKP,ANAKTUVUK PASS,68-08-01.0000N,151-44-36.0000W,68.133611,-151.7433333333333
259,ANC,PANC,TED STEVENS ANCHORAGE INTL,61-10-26.9634N,149-59-53.4791W,61.173889,-149.99805555555554
284,MRI,PAMR,MERRILL FIELD,61-12-48.7622N,149-50-41.0069W,61.213333,-149.84472222222223


#### Export dataframe to local csv

In [317]:
apt_final_df.set_index("icao_identifier", inplace=True)

In [318]:
apt_final_csv = "{0}//{1}".format(inputs_dir, "apt_formatted.csv")

In [319]:
apt_final_df.to_csv("{0}//{1}".format(inputs_dir, "apt_formatted.csv"))

### Data Load #1: ADDS (METARs)

METAR information should be added to the above features using ADDS data. METAR information is generated hourly. There are multiple search criteria available on the ADDS website. The stationString on in ADDS corresponds to the ICAO Identifier of the feature earlier identified. It is preferred that METAR information is available as both current and previous in the attribute data, in separate fields.

In [302]:
inputs_dir = r"C:\Users\albe9057\Documents\ANieto_SolutionEngineering\Projects\DOT\FAA\FAA_LivingAtlas_Layer\Inputs\ADDS_METARs_TAFs"; inputs_dir

'C:\\Users\\albe9057\\Documents\\ANieto_SolutionEngineering\\Projects\\DOT\\FAA\\FAA_LivingAtlas_Layer\\Inputs\\ADDS_METARs_TAFs'

In [303]:
def download_file(url, local_target_path):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_target_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_target_path

In [304]:
def xml_to_df(xml_data):
    tree = ET.parse(xml_data)
    root = tree.getroot()
    all_records = []
    for i, child in enumerate(root):
        if child.tag == "data":
            for subchild in child:
                if subchild.tag == "METAR":
                    record = {}
                    for subsubchild in subchild:
                        record[subsubchild.tag] = subsubchild.text
                    all_records.append(record)
    return pd.DataFrame(all_records)

In [328]:
def append_metar_for_icao_id(icao_val, apt_df, workspace_dir, hours_before_now=1, clean_up_temp_files=True):
    print("Processing METAR for '{0}'...".format(str(icao_val)))
    hours_before_now = 1
    metar_request_url = r"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&stationString={0}&hoursBeforeNow={1}&mostRecent=true".format(icao_val, hours_before_now)
    metars_temp_file = "{0}\\{1}".format(workspace_dir, "metar_{0}.xml".format(str(icao_val)))
    
    metars_temp_xml = download_file(metar_request_url, metars_temp_file) 

    metars_temp_df = xml_to_df(metars_temp_xml)
                                         
    metars_temp_df['icao_identifier'] = icao_val
                                         
    metars_temp_df.set_index("icao_identifier", inplace=True)
                                         
    apt_df = apt_df.join(metars_df)
    
    apt_df = apt_df.merge(metars_df)
    
    return apt_df

In [341]:
def compile_metars_for_all_icao_ids(icao_vals, workspace_dir, hours_before_now=1, clean_up_temp_files=True):
    # Set a container for each icao metar dataframe
    metars_panel = []
    
    for icao_val in icao_vals:
        
        try:
        
            print("Processing METAR for '{0}'...".format(str(icao_val)))
            hours_before_now = 1
            metar_request_url = r"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&stationString={0}&hoursBeforeNow={1}&mostRecent=true".format(icao_val, hours_before_now)
            metars_temp_file = "{0}\\{1}".format(workspace_dir, "metar_{0}.xml".format(str(icao_val)))

            metars_temp_xml = download_file(metar_request_url, metars_temp_file) 

            metars_temp_df = xml_to_df(metars_temp_xml)

            metars_temp_df['icao_identifier'] = icao_val

            metars_panel.append(metars_temp_df)
            
        except Exception as e:
            print("Error when attempting to download METAR for '{0}':".format(str(icao_val)))
            print(Exception)

    metars_final_df = pd.concat(metars_panel)
    
    metars_final_df.set_index("icao_identifier", inplace=True)

    return metars_final_df

In [333]:
# Get unique ICAO values into a list
icao_vals = apt_final_df.index.values.tolist() ; icao_vals

['PADK',
 'PAKH',
 'PFAK',
 'PAUT',
 'PAUK',
 'PFAL',
 'PAFM',
 'PAKP',
 'PANC',
 'PAMR',
 'PANI',
 'PANV',
 'PARC',
 'PAAK',
 'PATQ',
 'PABR',
 'PABA',
 'PAWB',
 'PABE',
 'PABT',
 'PAGQ',
 'PABV',
 'PFKT',
 'PABL',
 'PATW',
 'PACE',
 'PACI',
 'PALR',
 'PACK',
 'PFCB',
 'PAVA',
 'PAJC',
 'PACH',
 'PACR',
 'PFCL',
 'PACL',
 'PACD',
 'PACX',
 'PACV',
 'PASC',
 'PADE',
 'PADL',
 'PAEG',
 'PAEE',
 'PAII',
 'PFEL',
 'PAEM',
 'PAAN',
 'PAFA',
 'PAKF',
 'PAFL',
 'PFYU',
 'PAGB',
 'PAGA',
 'PAGM',
 'PAGL',
 'PAGX',
 'PAGK',
 'PAGS',
 'PAHN',
 'PAHV',
 'PAHC',
 'PAHO',
 'PAOH',
 'PAHP',
 'PAHU',
 'PAHL',
 'PAIG',
 'PAIL',
 'PAJN',
 'PAFE',
 'PALG',
 'PAKV',
 'PAKY',
 'PFKA',
 'PAEN',
 'PAKT',
 'PAIK',
 'PAVC',
 'PAKN',
 'PAKI',
 'PAVL',
 'PAKW',
 'PAOB',
 'PAKD',
 'PADQ',
 'PFKK',
 'PAJZ',
 'PADY',
 'PFKO',
 'PAOT',
 'PAKK',
 'PFKU',
 'PAKL',
 'PFKW',
 'PAGG',
 'PALB',
 'PAMX',
 'PAMC',
 'PAIN',
 'PAML',
 'PAMB',
 'PADM',
 'PAMY',
 'PAMD',
 'PAMH',
 'PAMO',
 'PANA',
 'PAPK',
 'PAOU',
 'PANN',
 

In [334]:
len(icao_vals)

2439

In [342]:
metars_df = compile_metars_for_all_icao_ids(icao_vals, inputs_dir, hours_before_now=1, clean_up_temp_files=True)

Processing METAR for 'PADK'...
Processing METAR for 'PAKH'...
Processing METAR for 'PFAK'...
Processing METAR for 'PAUT'...
Processing METAR for 'PAUK'...
Processing METAR for 'PFAL'...
Processing METAR for 'PAFM'...
Processing METAR for 'PAKP'...
Processing METAR for 'PANC'...
Processing METAR for 'PAMR'...
Processing METAR for 'PANI'...
Processing METAR for 'PANV'...
Processing METAR for 'PARC'...
Processing METAR for 'PAAK'...
Processing METAR for 'PATQ'...
Processing METAR for 'PABR'...
Processing METAR for 'PABA'...
Processing METAR for 'PAWB'...
Processing METAR for 'PABE'...
Processing METAR for 'PABT'...
Processing METAR for 'PAGQ'...
Processing METAR for 'PABV'...
Processing METAR for 'PFKT'...
Processing METAR for 'PABL'...
Processing METAR for 'PATW'...
Processing METAR for 'PACE'...
Processing METAR for 'PACI'...
Processing METAR for 'PALR'...
Processing METAR for 'PACK'...
Processing METAR for 'PFCB'...
Processing METAR for 'PAVA'...
Processing METAR for 'PAJC'...
Processi

,altim_in_hg,dewpoint_c,elevation_m,flight_category,latitude,longitude,maxT_c,metar_type,observation_time,precip_in,...,raw_text,sea_level_pressure_mb,sky_condition,station_id,temp_c,visibility_statute_mi,wind_dir_degrees,wind_gust_kt,wind_speed_kt,wx_string
icao_identifier,,,,,,,,,,,,,,,,,,,,,
PADK,29.949802,3.0,4.0,MVFR,51.88,-176.65,NaN,SPECI,2017-06-23T22:16:00Z,NaN,...,PADK 232216Z AUTO 18012KT 10SM BKN011 09/03 A2...,NaN,None,PADK,9.0,10.0,180,NaN,12,NaN
PAUT,30.351377,5.5,25.0,NaN,54.15,-165.6,NaN,METAR,2017-06-23T21:55:00Z,NaN,...,PAUT 232155Z AUTO 19011KT SCT033 10/06 A3035 R...,NaN,None,PAUT,9.6,NaN,190,NaN,11,NaN
PAFM,30.250984,5.6,88.0,VFR,67.1,-157.85,NaN,METAR,2017-06-23T21:56:00Z,NaN,...,PAFM 232156Z AUTO 19008KT 10SM OVC042 12/06 A3...,1025.7,None,PAFM,12.2,10.0,190,NaN,8,NaN
PAKP,30.121063,5.6,642.0,VFR,68.13,-151.73,NaN,METAR,2017-06-23T21:56:00Z,0.005,...,PAKP 232156Z AUTO 23010KT 9SM -RA FEW013 BKN05...,1022.5,None,PAKP,11.7,9.0,230,NaN,10,-RA
PANC,30.351377,7.8,38.0,VFR,61.17,-150.0,NaN,METAR,2017-06-23T21:53:00Z,NaN,...,PANC 232153Z VRB05KT 10SM SCT021 OVC034 12/08 ...,1027.7,None,PANC,12.2,10.0,0,NaN,5,NaN
PAMR,30.339567,8.9,41.0,MVFR,61.22,-149.85,NaN,METAR,2017-06-23T21:53:00Z,NaN,...,PAMR 232153Z 28005KT 10SM BKN018 OVC031 12/09 ...,1027.6,None,PAMR,12.2,10.0,280,NaN,5,NaN
PANI,30.268702,9.0,26.0,VFR,61.58,-159.53,NaN,SPECI,2017-06-23T22:16:00Z,NaN,...,PANI 232216Z AUTO 19005KT 10SM FEW090 18/09 A3...,NaN,None,PANI,18.0,10.0,190,NaN,5,NaN
PANV,30.239174,4.0,99.0,VFR,62.65,-160.18,NaN,SPECI,2017-06-23T22:16:00Z,NaN,...,PANV 232216Z AUTO 07007KT 10SM BKN085 19/04 A3...,NaN,None,PANV,19.0,10.0,70,NaN,7,NaN
PAAK,29.991142,4.0,17.0,IFR,52.22,-174.2,NaN,SPECI,2017-06-23T22:16:00Z,NaN,...,PAAK 232216Z 10016KT 1 1/4SM OVC005 07/04 A299...,NaN,None,PAAK,7.0,1.25,100,NaN,16,NaN


In [ ]:
# Merge metars to APT 
apt_final_df = apt_final_df.join(metars_df); apt_final_df

### Data Load #2: ADDS (TAFs)

TAF information should be added to the above features using ADDS data. TAF information is generated daily. There are multiple search criteria available on the ADDS website. The stationString on in ADDS corresponds to the ICAO Identifier of the feature earlier identified. Only the current TAF is required.

### Data Load #3: AIDAP NOTAM Data

#### Parse NOTAMs for closure: 

NOTAMs contain valuable information about the status of the airport in question. It would be exceptionally beneficial if the layer had an automatically derived Airport NOTAM Status field available. The proposed values for this field are:
•	Open
•	Closed
•	Limited

This is exceptionally complicated. The key term to search for in NOTAMs to identify if they will trigger this status is “AD AP CLSD”, however, the presence of “AD AP CLSD” does not necessarily trigger the field to full “Closed” status. The text immediately following “AD AP CLSD” will dictate if the status field receives a “Closed” or “Limited” status.

If, immediately following “AD AP CLSD” there is a space followed by a numeric string ten characters long, the airport receives a “Closed” status.

If, immediately following “AD AP CLSD” there is anything other than the previously mentioned string, the airport receives a “Limited” status.

 
Examples:

NOTAM:
!AGS 03/034 AGS AD AP CLSD TO FREIGHT, MEDICAL, AIR CARGO, AND FORMATION FLT 1704031200-1704112200
Automatic Status: Limited

NOTAM:
!WYS 11/004 WYS AD AP CLSD EXC LIFE FLT HEL 1611172345-1705152359EST
Automatic Status: Limited

NOTAM:
!FDR 03/002 FDR AD AP CLSD 1703281316-1704122359
Automatic Status: Closed

NOTAM:
!TEX 03/023 TEX AD AP CLSD 1704030300-1706251300
Automatic Status: Closed

An airport without a NOTAM containing “AD AP CLSD” receives the automatic status of “Open”.


### Data Load to GIS (In progress)

In [59]:
layer_properties = {'title':'Airports Living Atlas Test',
                    'description':'Test Airports Living Atlas Test for FAA',
                    'tags':'arcgis, python, faa, airports, notam'}

In [60]:
apt_csv_item = gis.content.add(item_properties=layer_properties, data=apt_final_csv)

In [61]:
apt_csv_item

<Item title:"Airports Living Atlas Test" type:CSV owner:anieto>

In [ ]:
apt_feature_layer_item = apt_csv_item.publish()

In [ ]:
apt_fc = gis.content.import_data(apt_final_df.head())